In [3]:
import psycopg

conn_v5 = psycopg.connect("postgresql://postgres@localhost:5432/dgidb")
conn_v4 = psycopg.connect("postgresql://postgres@localhost:5432/dgidb_v4")

In [24]:
import pandas as pd

### counts to get

* interactions
* gene claims
* drug claims
* gene categories

In [47]:
def make_dataframe(v5_counts, v4_counts):
    counts = pd.concat([pd.Series(v5_counts), pd.Series(v4_counts)], axis=1)
    counts = counts.fillna(0.0).astype(int)
    counts.columns = ["v5", "v4"]
    counts = counts.sort_index()
    return counts

### interactions

In [82]:
interactions_query = """
SELECT s.source_db_name, COUNT(1) FROM interaction_claims
LEFT JOIN sources s
ON interaction_claims.source_id = s.id
GROUP BY s.source_db_name
"""

with conn_v5.cursor() as cur:
    result = cur.execute(interactions_query).fetchall()
    v5_interaction_claims = {k: v for (k, v) in result}

with conn_v4.cursor() as cur:
    result = cur.execute(interactions_query).fetchall()
    v4_interaction_claims = {k: v for (k, v) in result}

In [83]:
v4_interaction_claims["ChEMBL"] = v4_interaction_claims["ChemblInteractions"]
del v4_interaction_claims["ChemblInteractions"]

In [84]:
interaction_claims = make_dataframe(v5_interaction_claims, v4_interaction_claims)
interaction_claims.to_csv("interaction_claims.csv")
interaction_claims

,v5,v4
CGI,368,372
CIViC,1083,959
COSMIC,37,37
CancerCommons,109,109
ChEMBL,16938,7610
ClearityFoundationBiomarkers,163,163
ClearityFoundationClinicalTrial,240,281
DTC,23879,23879
DoCM,76,76
FDA,427,427


### gene claims

TODO:

 * manually fill in DrugBank counts (from v4 paper?)
 * where is Entrez data? had to fill it in manually
 * get Guide to Pharmacology data

In [85]:
genes_query = """
SELECT s.source_db_name, COUNT(1) FROM gene_claims
LEFT JOIN sources s
ON gene_claims.source_id = s.id
GROUP BY s.source_db_name
"""

with conn_v5.cursor() as cur:
    result = cur.execute(genes_query).fetchall()
    v5_gene_claims = {k: v for (k, v) in result}

with conn_v4.cursor() as cur:
    result = cur.execute(genes_query).fetchall()
    v4_gene_claims = {k: v for (k, v) in result}

In [86]:
v4_gene_claims["BaderLab"] = v4_gene_claims["BaderLabGenes"]
del v4_gene_claims["BaderLabGenes"]
v4_gene_claims["ChEMBL"] = v4_gene_claims["ChemblInteractions"]
del v4_gene_claims["ChemblInteractions"]
v4_gene_claims["NCBI"] = 43741  # manually supply

In [87]:
gene_claims = make_dataframe(v5_gene_claims, v4_gene_claims)
gene_claims.to_csv("gene_claims.csv")
gene_claims

,v5,v4
BaderLab,48,300
CGI,123,118
CIViC,278,252
COSMIC,15,15
CancerCommons,48,48
CarisMolecularIntelligence,608,608
ChEMBL,1779,1085
ClearityFoundationBiomarkers,34,34
ClearityFoundationClinicalTrial,93,108
DTC,1016,1016


### drug claims

In [88]:
drug_query = """
SELECT s.source_db_name, COUNT(1) FROM drug_claims
LEFT JOIN sources s
ON drug_claims.source_id = s.id
GROUP BY s.source_db_name
"""

with conn_v5.cursor() as cur:
    result = cur.execute(drug_query).fetchall()
    v5_drug_claims = {k: v for (k, v) in result}

with conn_v4.cursor() as cur:
    result = cur.execute(drug_query).fetchall()
    v4_drug_claims = {k: v for (k, v) in result}

In [89]:
v4_drug_claims["ChEMBL"] = v4_drug_claims["ChemblDrugs"] + v4_drug_claims["ChemblInteractions"]
del v4_drug_claims["ChemblDrugs"]
del v4_drug_claims["ChemblInteractions"]

In [90]:
drug_claims = make_dataframe(v5_drug_claims, v4_drug_claims)
drug_claims.to_csv("drug_claims.csv")
drug_claims

,v5,v4
CGI,148,155
CIViC,416,367
COSMIC,28,28
CancerCommons,80,80
ChEMBL,5659,17037
ChemIDplus,5879,0
ClearityFoundationBiomarkers,64,64
ClearityFoundationClinicalTrial,114,115
DTC,6290,6290
DoCM,39,39
